<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/11_car_price/car_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ermittlung der Autokosten

Ein Gebrauchtwagenhändler entwickelt eine App zur Neukundengewinnung. Darin erfahren Sie schnell den Marktwert Ihres Autos. Zu Ihrer Verfügung stehen historische Daten: technische Daten, Ausstattung und Preise von Autos. Sie müssen ein Modell bauen, um die Kosten zu ermitteln.

Der Kunde ist wichtig:


*   Vorhersagequalität;
*   Vorhersagegeschwindigkeit;
*   Studienzeit.

# Datenaufbereitung

In [3]:
# Wir importieren die notwendigen Bibliotheken
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression

In [4]:
# Wir laden Tabellen mit Daten
df = pd.read_csv('/content/autos.csv')

In [5]:
# Wir werden die erhaltenen Daten studieren
df.info()
display(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

(354369, 16)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Wir werden Spalten entfernen, die für die Analyse nicht benötigt werden. Die preisbestimmenden Bedingungen sind unserer Meinung nach: Baujahr, Kilometerstand, Marke und Modell und ob das Auto repariert wurde. Es gibt viele Modelle, aber wir haben Daten über die Leistung von Autos. Um die Modellarbeit zu beschleunigen, nehmen Sie an, dass eine Marke nur ein Modell mit einer bestimmten Leistung haben kann, was auch den Autotyp, den Getriebetyp und den Kraftstofftyp bestimmt.

In [6]:
df.drop(['DateCrawled', 'VehicleType', 'RegistrationMonth', 'DateCreated', 'NumberOfPictures', 'PostalCode', 
         'LastSeen', 'Model', 'Gearbox', 'FuelType'], axis=1, inplace = True)

display(df.info())
display(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354369 non-null  int64 
 1   RegistrationYear  354369 non-null  int64 
 2   Power             354369 non-null  int64 
 3   Kilometer         354369 non-null  int64 
 4   Brand             354369 non-null  object
 5   NotRepaired       283215 non-null  object
dtypes: int64(4), object(2)
memory usage: 16.2+ MB


None

(354369, 6)

,Price,RegistrationYear,Power,Kilometer,Brand,NotRepaired
0,480,1993,0,150000,volkswagen,NaN
1,18300,2011,190,125000,audi,yes
2,9800,2004,163,125000,jeep,NaN
3,1500,2001,75,150000,volkswagen,no
4,3600,2008,69,90000,skoda,no


In [7]:
# Wir werden kategoriale Merkmale durch numerische ersetzen
df_ohe = pd.get_dummies(df, drop_first=True)
display(df_ohe.shape)
display(df_ohe.head())

(354369, 44)

,Price,RegistrationYear,Power,Kilometer,Brand_audi,Brand_bmw,Brand_chevrolet,Brand_chrysler,Brand_citroen,Brand_dacia,...,Brand_skoda,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
0,480,1993,0,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,18300,2011,190,125000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75,150000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,3600,2008,69,90000,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
